In [7]:
# --- Бібліотеки для даної роботи ---
try:
    import numpy, pandas, plotly, nbformat
    print("Бібліотеки вже встановлені. Пропускаємо інсталяцію.")
except ImportError:
    print("Встановлюємо бібліотеки...")
    %pip install numpy pandas plotly nbformat

Бібліотеки вже встановлені. Пропускаємо інсталяцію.


## Домашнє завдання: Тема 3. Векторні операції в n-вимірних просторах

### **Це допоможе закріпити такі навички:**

- Векторний, скалярний добуток
- Застосування функції Panda, Numpy

### **Завдання (крок за кроком):**

***Для цієї задачі використайте наданий файл з NLP моделлю.***

1. **Визначте DataFrame з тривимірними векторами слів:**
   - Завантажте модель word embeddings, використовуючи файл, який містить NLP модель.
   - Витягніть тривимірні вектори для всіх слів з цієї моделі.
   - Створіть DataFrame, в якому буде міститися інформація про слова та їхні тривимірні вектори.

2. **Визначте функції для пошуку найближчого слова:**
   - Напишіть функцію, яка приймає тривимірний вектор та знаходить найближче слово в моделі word embeddings_subset.
   - Використайте цю функцію для кількох прикладів та переконайтеся, що результати є коректними.

3. **Обчисліть векторний добуток для знаходження ортогонального слова:**
   - Виберіть декілька довільних пар слів.
   - Обчисліть векторний добуток для кожної пари слів та використайте раніше написану функцію для знаходження найближчого слова.
   - Проаналізуйте результати та спробуйте їх інтерпретувати.

4. **Напишіть функції визначення кута між словами:**
   - Розробіть функцію, яка обчислює кут між векторами для довільних двох слів.
   - Протестируйте цю функцію для різних пар слів.
   - Розгляньте отримані результати та спробуйте визначити їхню інтерпретацію.

5.  **Висновок:**

      Використовуйте середовище Jupyter Notebook або іншу платформу для написання текстових висновків та аналізу результатів.

**0. Імпорт необхідних бібліотек:**

In [13]:
import warnings
warnings.filterwarnings('ignore')

import pickle
import numpy as np
import pandas as pd
import plotly.graph_objects as go

**1. Визначте DataFrame з тривимірними векторами слів:**

In [9]:
with open('word_embeddings_subset.p', 'rb') as f:
    full_embeddings = pickle.load(f)

word_embeddings = {word: np.array(vec[:3]) for word, vec in full_embeddings.items()}

df_embeddings = pd.DataFrame.from_dict(word_embeddings, orient='index', columns=['x', 'y', 'z'])
df_embeddings.index.name = 'Слово'
df_embeddings.reset_index(inplace=True)

available_words = df_embeddings['Слово'].tolist()

fig_3d = go.Figure()

fig_3d.add_trace(go.Scatter3d(
    x=df_embeddings['x'][15:],
    y=df_embeddings['y'][15:],
    z=df_embeddings['z'][15:],
    mode='markers+text',
    text=df_embeddings['Слово'][15:],
    textposition='top center',
    textfont=dict(color='rgba(255, 255, 255, 0.3)', size=10),
    marker=dict(
        size=5, 
        color=df_embeddings['z'][15:], 
        colorscale='Plasma', 
        opacity=0.4
    ),
    name='Інші слова',
    hoverinfo='text'
))

fig_3d.add_trace(go.Scatter3d(
    x=df_embeddings['x'][:15],
    y=df_embeddings['y'][:15],
    z=df_embeddings['z'][:15],
    mode='markers+text',
    text=["<b>" + w + "</b>" for w in df_embeddings['Слово'][:15]],
    textposition='top center',
    textfont=dict(color='#ffea00', size=16),
    marker=dict(
        size=12,
        color='#00ffcc',
        line=dict(width=2, color='white'),
        opacity=1.0
    ),
    name='Топ-15 (з таблиці)',
    hoverinfo='text'
))

fig_3d.update_layout(
    title='Інтерактивний 3D простір слів (Топ-15 виділено)',
    template='plotly_dark',
    height=900,
    width=1000,
    margin=dict(l=0, r=0, b=0, t=50),
    scene=dict(
        xaxis=dict(title='X (Вимір 1)', backgroundcolor="rgb(20, 24, 34)", gridcolor="#444"),
        yaxis=dict(title='Y (Вимір 2)', backgroundcolor="rgb(20, 24, 34)", gridcolor="#444"),
        zaxis=dict(title='Z (Вимір 3)', backgroundcolor="rgb(20, 24, 34)", gridcolor="#444"),
    ),
    showlegend=True,
    legend=dict(x=0.02, y=0.95, bgcolor='rgba(0,0,0,0.5)', font=dict(color='white'))
)

df_head = df_embeddings.head(15)

fig_table = go.Figure(data=[go.Table(
    columnwidth=[40, 100, 100, 100, 100], 
    header=dict(
        values=["<b>№</b>"] + [f"<b>{col}</b>" for col in df_embeddings.columns],
        fill_color='#2c2c2c',
        font=dict(color='white', size=14),
        align='center',
        height=35
    ),
    cells=dict(
        values=[
            list(range(1, len(df_head) + 1)), 
            df_head['Слово'],
            df_head['x'].round(4),
            df_head['y'].round(4),
            df_head['z'].round(4)
        ],
        fill_color='#111111',
        font=dict(color='lightgrey', size=13),
        align=['center', 'left', 'center', 'center', 'center'],
        height=30
    )
)])

fig_table.update_layout(
    title='Таблиця 3D векторів слів (перші 15)',
    template='plotly_dark',
    margin=dict(l=0, r=0, t=40, b=0),
    height=550 
)

print("Красивий Вивід:")
fig_3d.show()
fig_table.show()

print("Технічний вивід:")
print(df_embeddings.head(15).to_dict(orient='records'))
print(f"Розмірність DataFrame: {df_embeddings.shape}")
print(f"Всього слів у файлі: {len(available_words)}")
print(f"Ось вони: {available_words}")

Красивий Вивід:


Технічний вивід:
[{'Слово': 'country', 'x': -0.080078125, 'y': 0.1337890625, 'z': 0.1435546875}, {'Слово': 'city', 'x': -0.01007080078125, 'y': 0.057373046875, 'z': 0.18359375}, {'Слово': 'China', 'x': -0.0732421875, 'y': 0.1357421875, 'z': 0.10888671875}, {'Слово': 'Iraq', 'x': 0.19140625, 'y': 0.125, 'z': -0.0654296875}, {'Слово': 'oil', 'x': -0.1396484375, 'y': 0.062255859375, 'z': -0.279296875}, {'Слово': 'town', 'x': 0.12353515625, 'y': 0.1591796875, 'z': 0.030029296875}, {'Слово': 'Canada', 'x': -0.13671875, 'y': -0.154296875, 'z': 0.26953125}, {'Слово': 'London', 'x': -0.267578125, 'y': 0.0927734375, 'z': -0.23828125}, {'Слово': 'England', 'x': -0.1982421875, 'y': 0.115234375, 'z': 0.0625}, {'Слово': 'Australia', 'x': 0.048828125, 'y': -0.1943359375, 'z': -0.04150390625}, {'Слово': 'Japan', 'x': 0.05078125, 'y': 0.25, 'z': 0.1669921875}, {'Слово': 'Pakistan', 'x': -0.30859375, 'y': 0.0291748046875, 'z': 0.185546875}, {'Слово': 'Iran', 'x': -0.126953125, 'y': 0.1650390625, 'z': 0

**2. Визначте функції для пошуку найближчого слова:**

In [10]:
def find_closest_word(target_vector, embeddings_dict):
    closest_word = None
    min_distance = float('inf')
    
    for word, vec in embeddings_dict.items():
        distance = np.linalg.norm(np.array(target_vector) - np.array(vec))
        if distance < min_distance:
            min_distance = distance
            closest_word = word
            
    return closest_word, min_distance

test_word = available_words[0] 
test_vector = np.array(word_embeddings[test_word])
closest_exact, dist_exact = find_closest_word(test_vector, word_embeddings)

offset_vector = np.array([0.1, -0.05, 0.2])
test_vector_offset = test_vector + offset_vector
closest_offset, dist_offset = find_closest_word(test_vector_offset, word_embeddings)

df_results = pd.DataFrame({
    "Тестовий вектор": [f"Точний вектор '{test_word}'", f"Вектор '{test_word}' + зміщення"],
    "Знайдене слово": [closest_exact, closest_offset],
    "Евклідова відстань": [round(dist_exact, 4), round(dist_offset, 4)]
})

fig_res_table = go.Figure(data=[go.Table(
    columnwidth=[200, 150, 150],
    header=dict(
        values=[f"<b>{col}</b>" for col in df_results.columns],
        fill_color='#2c2c2c',
        font=dict(color='white', size=14),
        align='center',
        height=35
    ),
    cells=dict(
        values=[df_results[col] for col in df_results.columns],
        fill_color='#111111',
        font=dict(color='lightgrey', size=13),
        align=['left', 'center', 'center'],
        height=30
    )
)])

fig_res_table.update_layout(
    title='Результати пошуку найближчого слова',
    template='plotly_dark',
    margin=dict(l=0, r=0, t=40, b=0),
    height=150
)

vec_closest = np.array(word_embeddings[closest_offset])
fig_mini_3d = go.Figure()

fig_mini_3d.add_trace(go.Scatter3d(
    x=[test_vector[0]], y=[test_vector[1]], z=[test_vector[2]],
    mode='markers+text', text=[f"<b>{test_word}</b> (Старт)"],
    textposition='top center', textfont=dict(color='#ffea00', size=14),
    marker=dict(size=10, color='#ffea00'), name='Оригінальне слово'
))

fig_mini_3d.add_trace(go.Scatter3d(
    x=[test_vector_offset[0]], y=[test_vector_offset[1]], z=[test_vector_offset[2]],
    mode='markers+text', text=["<b>Точка зміщення</b>"],
    textposition='top right', textfont=dict(color='#ff3366', size=12),
    marker=dict(size=6, color='#ff3366', symbol='x'), name='Шукана точка'
))

fig_mini_3d.add_trace(go.Scatter3d(
    x=[vec_closest[0]], y=[vec_closest[1]], z=[vec_closest[2]],
    mode='markers+text', text=[f"<b>{closest_offset}</b> (Фініш)"],
    textposition='bottom center', textfont=dict(color='#00ffcc', size=14),
    marker=dict(size=10, color='#00ffcc'), name='Знайдене слово'
))

fig_mini_3d.add_trace(go.Scatter3d(
    x=[test_vector[0], test_vector_offset[0], vec_closest[0]],
    y=[test_vector[1], test_vector_offset[1], vec_closest[1]],
    z=[test_vector[2], test_vector_offset[2], vec_closest[2]],
    mode='lines', line=dict(color='white', width=2, dash='dot'),
    name='Векторний шлях'
))

fig_mini_3d.update_layout(
    title='3D Візуалізація: Векторне зміщення та пошук найближчого сусіда',
    template='plotly_dark',
    height=500,
    margin=dict(l=0, r=0, b=0, t=40),
    scene=dict(
        xaxis=dict(title='X', backgroundcolor="rgb(20, 24, 34)", gridcolor="#444"),
        yaxis=dict(title='Y', backgroundcolor="rgb(20, 24, 34)", gridcolor="#444"),
        zaxis=dict(title='Z', backgroundcolor="rgb(20, 24, 34)", gridcolor="#444")
    )
)

print("Красивий Вивід:")
fig_mini_3d.show()
fig_res_table.show()

print("Технічний вивід:")
print(f"Ціль 1 (точний збіг): знайдено слово='{closest_exact}', відстань={dist_exact}")
print(f"Ціль 2 (зі зміщенням): знайдено слово='{closest_offset}', відстань={dist_offset}")

Красивий Вивід:


Технічний вивід:
Ціль 1 (точний збіг): знайдено слово='country', відстань=0.0
Ціль 2 (зі зміщенням): знайдено слово='France', відстань=0.03495974405478708


**3. Обчисліть векторний добуток для знаходження ортогонального слова:**

In [32]:
idx1 = 9
idx2 = 55

if idx1 >= len(available_words) or idx2 >= len(available_words):
    print("❌ Помилка: Введено індекс, який перевищує розмір словника!")
    print(f"Доступні індекси: від 0 до {len(available_words) - 1}. Будь ласка, змініть значення.")

elif idx1 == idx2 or available_words[idx1] == available_words[idx2]:
    print(f"⚠️ Попередження: Ви вибрали однакове слово '{available_words[idx1]}' двічі!")
    print("Математично: Векторний добуток вектора самого на себе завжди дорівнює [0. 0. 0.].")
    print("Неможливо знайти перпендикуляр. Будь ласка, виберіть різні слова.")

else:
    word1, word2 = available_words[idx1], available_words[idx2]
    vec1 = np.array(word_embeddings[word1])
    vec2 = np.array(word_embeddings[word2])

    cross_product = np.cross(vec1, vec2)

    if np.allclose(cross_product, [0, 0, 0]):
        print(f"⚠️ Попередження: Вектори слів '{word1}' та '{word2}' паралельні. Їх векторний добуток — нуль.")
    else:
        orthogonal_word, ortho_dist = find_closest_word(cross_product, word_embeddings)
        vec_ortho = np.array(word_embeddings[orthogonal_word])

        df_cross = pd.DataFrame({
            "Слово 1": [word1],
            "Слово 2": [word2],
            "Векторний добуток (X, Y, Z)": [str(np.round(cross_product, 4))],
            "Знайдене слово": [orthogonal_word],
            "Відстань": [round(ortho_dist, 4)]
        })

        fig_cross_table = go.Figure(data=[go.Table(
            columnwidth=[100, 100, 250, 150, 100],
            header=dict(
                values=[f"<b>{col}</b>" for col in df_cross.columns],
                fill_color='#2c2c2c',
                font=dict(color='white', size=14),
                align='center',
                height=35
            ),
            cells=dict(
                values=[df_cross[col] for col in df_cross.columns],
                fill_color='#111111',
                font=dict(color='lightgrey', size=13),
                align='center',
                height=30
            )
        )])

        fig_cross_table.update_layout(
            title='Результати: Знаходження ортогонального слова',
            template='plotly_dark',
            margin=dict(l=0, r=0, t=40, b=0),
            height=120
        )

        fig_cross_3d = go.Figure()
        origin = [0, 0, 0] # Початок координат

        def add_vector_trace(fig, vec, name, color, text_pos):
            fig.add_trace(go.Scatter3d(
                x=[origin[0], vec[0]], y=[origin[1], vec[1]], z=[origin[2], vec[2]],
                mode='lines+markers+text',
                text=["", f"<b>{name}</b>"],
                textposition=text_pos,
                textfont=dict(color=color, size=14),
                marker=dict(size=[0, 6], color=color),
                line=dict(color=color, width=5),
                name=name
            ))

        add_vector_trace(fig_cross_3d, vec1, f"Вектор 1 ({word1})", '#ff3366', 'bottom left')
        add_vector_trace(fig_cross_3d, vec2, f"Вектор 2 ({word2})", '#33ccff', 'bottom right')
        add_vector_trace(fig_cross_3d, cross_product, 'Векторний добуток', '#ffea00', 'top center')

        fig_cross_3d.add_trace(go.Scatter3d(
            x=[vec_ortho[0]], y=[vec_ortho[1]], z=[vec_ortho[2]],
            mode='markers+text',
            text=[f"<b>{orthogonal_word}</b> (Знайдене)"],
            textposition='middle right',
            textfont=dict(color='#00ffcc', size=14),
            marker=dict(size=10, color='#00ffcc', symbol='diamond'),
            name='Знайдене слово'
        ))

        fig_cross_3d.add_trace(go.Scatter3d(
            x=[cross_product[0], vec_ortho[0]],
            y=[cross_product[1], vec_ortho[1]],
            z=[cross_product[2], vec_ortho[2]],
            mode='lines',
            line=dict(color='white', width=2, dash='dot'),
            name='Відстань (Похибка)'
        ))

        fig_cross_3d.update_layout(
            title='3D Векторний простір: Перпендикулярність (Ортогональність)',
            template='plotly_dark',
            height=600,
            margin=dict(l=0, r=0, b=0, t=40),
            scene=dict(
                xaxis=dict(title='X', backgroundcolor="rgb(20, 24, 34)", gridcolor="#444"),
                yaxis=dict(title='Y', backgroundcolor="rgb(20, 24, 34)", gridcolor="#444"),
                zaxis=dict(title='Z', backgroundcolor="rgb(20, 24, 34)", gridcolor="#444")
            )
        )

        print("Красивий Вивід:")
        fig_cross_3d.show()
        fig_cross_table.show()

        print("Технічний вивід:")
        print(f"Вектор 1 ('{word1}'): {np.round(vec1, 4)}")
        print(f"Вектор 2 ('{word2}'): {np.round(vec2, 4)}")
        print(f"Масив векторного добутку: {np.round(cross_product, 4)}")
        print(f"Найближче ортогональне слово: '{orthogonal_word}' з відстанню {round(ortho_dist, 4)}")

Красивий Вивід:


Технічний вивід:
Вектор 1 ('Australia'): [ 0.0488 -0.1943 -0.0415]
Вектор 2 ('Berlin'): [0.1416 0.252  0.0262]
Масив векторного добутку: [ 0.0054 -0.0072  0.0398]
Найближче ортогональне слово: 'Stockholm' з відстанню 0.019899999722838402


**4. Напишіть функції визначення кута між словами:**

In [12]:
def calculate_angle(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    
    if norm1 == 0 or norm2 == 0:
        return 0.0
        
    cos_theta = dot_product / (norm1 * norm2)
    cos_theta = np.clip(cos_theta, -1.0, 1.0)
    
    angle_rad = np.arccos(cos_theta)
    return np.degrees(angle_rad)

word_pairs = [('happy', 'sad'), ('king', 'queen'), ('boy', 'girl'), ('up', 'down'), ('good', 'bad'), ('country', 'town')]

if 'available_words' in locals() and len(available_words) >= 2:
    word_pairs = [(available_words[0], available_words[1]), (available_words[1], available_words[2])] + word_pairs

results_angles = []
valid_pairs = []

for w1, w2 in word_pairs:
    if w1 in word_embeddings and w2 in word_embeddings:
        v1 = np.array(word_embeddings[w1])
        v2 = np.array(word_embeddings[w2])
        angle = calculate_angle(v1, v2)
        results_angles.append((w1, w2, round(angle, 2)))
        valid_pairs.append((w1, w2, v1, v2, angle))
    else:
        missing = [w for w in (w1, w2) if w not in word_embeddings]
        results_angles.append((w1, w2, f"Не знайдено: {', '.join(missing)}"))

df_angles = pd.DataFrame(results_angles, columns=["Слово 1", "Слово 2", "Кут між векторами (градуси)"])

fig_angle_table = go.Figure(data=[go.Table(
    columnwidth=[150, 150, 250],
    header=dict(
        values=[f"<b>{col}</b>" for col in df_angles.columns],
        fill_color='#2c2c2c',
        font=dict(color='white', size=14),
        align='center',
        height=35
    ),
    cells=dict(
        values=[df_angles[col] for col in df_angles.columns],
        fill_color='#111111',
        font=dict(color='lightgrey', size=13),
        align='center',
        height=30
    )
)])

fig_angle_table.update_layout(
    title='Результати: Кути між векторами слів',
    template='plotly_dark',
    margin=dict(l=0, r=0, t=40, b=0),
    height=250
)

fig_angle_3d = go.Figure()

if valid_pairs:
    w1, w2, vec1, vec2, angle = valid_pairs[0]
    origin = [0, 0, 0]
    
    fig_angle_3d.add_trace(go.Scatter3d(
        x=[origin[0], vec1[0]], y=[origin[1], vec1[1]], z=[origin[2], vec1[2]],
        mode='lines+markers+text',
        text=["", f"<b>{w1}</b>"],
        textposition='top center',
        textfont=dict(color='#00ffcc', size=16),
        marker=dict(size=[0, 8], color='#00ffcc'),
        line=dict(color='#00ffcc', width=6),
        name=f'Вектор 1 ({w1})'
    ))
    
    fig_angle_3d.add_trace(go.Scatter3d(
        x=[origin[0], vec2[0]], y=[origin[1], vec2[1]], z=[origin[2], vec2[2]],
        mode='lines+markers+text',
        text=["", f"<b>{w2}</b>"],
        textposition='top center',
        textfont=dict(color='#ff3366', size=16),
        marker=dict(size=[0, 8], color='#ff3366'),
        line=dict(color='#ff3366', width=6),
        name=f'Вектор 2 ({w2})'
    ))

    fig_angle_3d.add_trace(go.Scatter3d(
        x=[vec1[0], vec2[0]], y=[vec1[1], vec2[1]], z=[vec1[2], vec2[2]],
        mode='lines',
        line=dict(color='rgba(255, 255, 255, 0.5)', width=3, dash='dash'),
        name=f'Кут: {round(angle, 1)}°'
    ))

    fig_angle_3d.update_layout(
        title=f'3D Візуалізація кута: "{w1}" та "{w2}" (Кут: {round(angle, 1)}°)',
        template='plotly_dark',
        height=600,
        margin=dict(l=0, r=0, b=0, t=40),
        scene=dict(
            xaxis=dict(title='X', backgroundcolor="rgb(20, 24, 34)", gridcolor="#444"),
            yaxis=dict(title='Y', backgroundcolor="rgb(20, 24, 34)", gridcolor="#444"),
            zaxis=dict(title='Z', backgroundcolor="rgb(20, 24, 34)", gridcolor="#444")
        )
    )

print("Красивий Вивід:")
if valid_pairs:
    fig_angle_3d.show()
fig_angle_table.show()

print("Технічний вивід:")
for res in results_angles:
    if "Не знайдено" in str(res[2]):
        print(f"Пара: ('{res[0]}', '{res[1]}') -> {res[2]}")
    else:
        print(f"Пара: ('{res[0]}', '{res[1]}') -> Кут: {res[2]} градусів")

Красивий Вивід:


Технічний вивід:
Пара: ('country', 'city') -> Кут: 31.420000076293945 градусів
Пара: ('city', 'China') -> Кут: 38.36000061035156 градусів
Пара: ('happy', 'sad') -> Кут: 77.12000274658203 градусів
Пара: ('king', 'queen') -> Кут: 101.73999786376953 градусів
Пара: ('boy', 'girl') -> Не знайдено: boy, girl
Пара: ('up', 'down') -> Не знайдено: up, down
Пара: ('good', 'bad') -> Не знайдено: good, bad
Пара: ('country', 'town') -> Кут: 68.66000366210938 градусів


**5. Висновок:**

### Аналітичні обчислення та висновки

#### 1. Обчислення векторного добутку аналітично

Векторний добуток двох векторів $\vec{a} = (a_x, a_y, a_z)$ та $\vec{b} = (b_x, b_y, b_z)$ у тривимірному просторі знаходиться через визначник матриці, де $\mathbf{i}, \mathbf{j}, \mathbf{k}$ — одиничні вектори базису:

$$\vec{a} \times \vec{b} = \begin{vmatrix} \mathbf{i} & \mathbf{j} & \mathbf{k} \\ a_x & a_y & a_z \\ b_x & b_y & b_z \end{vmatrix}$$

Розкриваючи визначник по першому рядку, отримуємо проміжні кроки:

$$\vec{a} \times \vec{b} = \mathbf{i}(a_y b_z - a_z b_y) - \mathbf{j}(a_x b_z - a_z b_x) + \mathbf{k}(a_x b_y - a_y b_x)$$

У контексті NLP (Word Embeddings) векторний добуток дає вектор, який є **ортогональним (перпендикулярним)** до площини, що утворена двома вихідними словами. Знайдене найближче слово до цього нового вектора зазвичай представляє концепцію, яка семантично віддалена або є незалежною від обох початкових слів (ортогональність у лінійній алгебрі часто відповідає відсутності кореляції у статистиці).

#### 2. Обчислення кута між векторами (Косинусна подібність)

Кут $\theta$ між двома векторами обчислюється за допомогою скалярного добутку. В обробці природної мови ця метрика відома як **косинусна подібність (Cosine Similarity)**. Формула має вигляд:

$$\cos(\theta) = \frac{\vec{a} \cdot \vec{b}}{|\vec{a}| |\vec{b}|}$$

Проміжні кроки для розрахунку:

1. Знаходимо скалярний добуток: $\vec{a} \cdot \vec{b} = a_x b_x + a_y b_y + a_z b_z$
2. Знаходимо довжини (норми) векторів: $|\vec{a}| = \sqrt{a_x^2 + a_y^2 + a_z^2}$ та $|\vec{b}| = \sqrt{b_x^2 + b_y^2 + b_z^2}$
3. Обчислюємо арккосинус для знаходження кута в радіанах: $\theta = \arccos(\cos(\theta))$
4. Переводимо в градуси: $\theta_{\text{deg}} = \theta \cdot \frac{180}{\pi}$

#### 3. Висновки та інтерпретація результатів

Аналіз кутів між словами в моделі Word Embeddings показує рівень їхньої семантичної близькості:

* **Синоніми та споріднені слова:** Зазвичай мають найменший кут між своїми векторами (косинус наближається до 1), оскільки вони часто зустрічаються в однакових контекстах.
* **Антоніми:** Цікавою особливістю NLP-моделей є те, що антоніми (наприклад, "good" і "bad") також часто мають відносно невеликий кут. Це пов'язано з тим, що вони граматично і контекстуально взаємозамінні в реченнях.
* **Непов'язані слова:** Утворюють кути, близькі до 90 градусів (ортогональність, косинус наближається до 0), що свідчить про відсутність спільних контекстів.
* **Критичне зауваження щодо розмірності:** Варто зазначити, що у повноцінних сучасних NLP моделях вектори слів (embeddings) зазвичай мають від 100 до 300 вимірів. У даній роботі ми використали лише перші три виміри (x, y, z) для можливості 3D-візуалізації. Таке жорстке усічення (Truncation) неминуче призводить до втрати значної частини семантичної інформації. Відповідно, метрики відстаней та кутів у нашому 3D-просторі є лише наближеними і слугують математичною демонстрацією принципів векторного аналізу, а не точним відображенням лінгвістичних зв'язків оригінальної моделі.